In [ ]:
# Find the latest version of spark from https://downloads.apache.org/spark/
# https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz


# Install Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download and install Apache Spark
# Ensure this URL points to a valid Spark tarball
spark_version = '3.5.0'
hadoop_version = '3'
spark_filename = f'spark-{spark_version}-bin-hadoop{hadoop_version}'
spark_url = f'https://downloads.apache.org/spark/spark-{spark_version}/{spark_filename}.tgz'

!wget -q {spark_url}
!tar xf {spark_filename}.tgz

# Install findspark
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_filename}"

# Initialize findspark to find Spark
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header = True)
# 2. Create a temporary view of the DataFrame.
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
fourbeds_df = df.filter(df['bedrooms'] == 4)

result_df = fourbeds_df.groupBy('date_built').agg(round(avg('price'), 2).alias('average_price')).orderBy('date_built')

result_df.show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    296800.75|
|      2011|     302141.9|
|      2012|    298233.42|
|      2013|    299999.39|
|      2014|    299073.89|
|      2015|    307908.86|
|      2016|    296050.24|
|      2017|    296576.69|
+----------+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
questionFour_df = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3))

result_df = questionFour_df.groupBy('date_built').agg(round(avg('price'), 2).alias('average_price')).orderBy('date_built')

result_df.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

questionFive_df = df.filter((df['bedrooms'] == 3) &
                        (df['bathrooms'] == 3) &
                        (df['floors'] == 2) &
                        (df['sqft_living'] >= 2000))

result_df = questionFive_df.groupBy('date_built').agg(round(avg('price'), 2).alias('average_price')).orderBy('date_built')

result_df.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

questionSix_df = df.filter(df['price'] >= 350000)

result_df = questionSix_df.agg(round(avg('price'), 2).alias('average_price'), round(avg('view'), 2).alias('average_view'))

result_df.show()


start_time = time.time()
result_df.collect()
runtime = time.time() - start_time
print("--- %s seconds ---" % runtime)

+-------------+------------+
|average_price|average_view|
+-------------+------------+
|    292003.61|        25.9|
+-------------+------------+

--- 0.5712506771087646 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
df.createOrReplaceTempView("home_sales")


spark.catalog.cacheTable("home_sales")


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [75]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result_df_cached = spark.sql("""
    SELECT
        ROUND(AVG(price), 2) AS average_price,
        ROUND(AVG(view), 2) AS average_view
    FROM
        home_sales
    WHERE
        price >= 350000
""")

result_df_cached.show()

runtime_cached = time.time() - start_time
print("--- Cached Runtime: %s seconds ---" % runtime_cached)

spark.catalog.uncacheTable("home_sales")



+-------------+------------+
|average_price|average_view|
+-------------+------------+
|    473796.26|       32.26|
+-------------+------------+

--- Cached Runtime: 0.2679941654205322 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("formatted_home_sales_partitioned.parquet")


In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("formatted_home_sales_partitioned.parquet")


In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
result_df = parquet_df.filter(parquet_df['price'] >= 350000).agg(
    round(avg('price'), 2).alias('average_price'),
    round(avg('view'), 2).alias('average_view')
)

result_df.show()



runtime = time.time() - start_time
print("--- Parquet Runtime: %s seconds ---" % runtime)

+-------------+------------+
|average_price|average_view|
+-------------+------------+
|    473796.26|       32.26|
+-------------+------------+

--- Parquet Runtime: 0.7502014636993408 seconds ---


In [85]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [86]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

if is_cached:
    print("home_sales is still cached.")
else:
    print("home_sales is not cached.")

home_sales is not cached.
